In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install -U cloudpickle
!pip install imbalanced-learn

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [ ]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [ ]:
df = 'FRAUD_DETECTION_ANALY_MASTER_TABLE'

In [ ]:
sf_df = my_session.sql("select * from {}".format(df))

In [ ]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [ ]:
print(pandas_df.isnull().sum())

In [ ]:
Original_df = pandas_df.copy()

In [ ]:
Original_df

In [ ]:
df = Original_df.drop(["TRANSACTION_ID", "CUSTOMER_ID", "DOB", "NAME", "CITY","COUNTY","STATE","YEAR","QUARTER","MONTH","AGE_GROUP","TRANSACTION_DATE","ACCOUNT_OPENING_DATE","LAST_LOGIN","TIME_OF_DAY","DAY_OF_WEEK","TIME","TIME_SINCE_LAST_TRANSACTION","FRAUD_FLAG"], axis = 1)

In [ ]:
CATEGORICAL_COLUMNS = ["MERCHANT_CATEGORY","PAYMENT_METHOD","CUSTOMER_SEGMENT","ACCOUNT_TYPE","DEVICE_USED","TRANSACTION_STATUS","CROSS_BORDER_TRANSACTION_INDICATOR","NEW_DEVICE_INDICATOR","GENDER","ANAMOLY_RISK_CATEGORY","MERCHANT_RISK_CATEGORY","CUSTOMER_LOYALTY_CATEGORY","TRANSACTION_TYPE"]
NUMERICAL_COLUMNS = ["ANOMALY_SCORE","TRANSACTION_AMOUNT","CREDIT_LIMIT","AGE_OF_ACCOUNT","FREQUENCY_OF_TRANSACTIONS","MERCHANT_RISK_SCORE","CUSTOMER_LOYALTY_SCORE","DISTANCE_FROM_HOME_ADDRESS","DISTANCE_FROM_LAST_TRANSACTION","NUMBER_OF_REFUNDS","NUMBER_OF_CHARGEBACKS","CREDIT_UTILIZATION_RATIO",
                     "CHANGE_IN_SPENDING_BEHAVIOR","TRANSACTION_VELOCITY","AGE","SUSPICIOUS_FLAG","PREVIOUS_FRAUD_REPORTS"]
LABEL_COLUMNS = ["FRAUD_INDICATOR"]
DROPPED_COLUMNS = ["TRANSACTION_ID", "CUSTOMER_ID", "DOB", "NAME", "CITY","COUNTY","STATE","YEAR","QUARTER","MONTH","AGE_GROUP","TRANSACTION_DATE","ACCOUNT_OPENING_DATE","LAST_LOGIN","TIME_OF_DAY","DAY_OF_WEEK","TIME","TIME_SINCE_LAST_TRANSACTION","FRAUD_FLAG"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [ ]:
# Filter feature columns
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS
feature_columns = [col for col in feature_columns if col in Original_df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in Original_df.columns]
 
# Split data into features and labels
X = Original_df[feature_columns + DROPPED_COLUMNS]
y = Original_df[LABEL_COLUMNS].values.ravel()  # Flatten to 1D array for consistency

In [ ]:
# Preprocessing for numerical data
from sklearn.preprocessing import StandardScaler
numerical_transformer = StandardScaler()

In [ ]:
# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [ ]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, NUMERICAL_COLUMNS),
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS)
    ])

In [ ]:
# Create and train the logistic regression model within a pipeline
from sklearn.linear_model import LogisticRegression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', LogisticRegression())])


In [ ]:
# splitting data into training set and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape,y_test.shape

In [ ]:
# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named 'df' and has a column 'FraudIndicator'
# Create a count plot for the 'FraudIndicator' column
plt.figure(figsize=(8, 6))  # Optional: Adjust the figure size
sns.countplot(data=Original_df, x='FRAUD_INDICATOR', palette='viridis')
plt.title('Count Plot of Fraud Indicator')
plt.xlabel('Fraud Indicator')
plt.ylabel('Count')
plt.show()


In [ ]:
!pip install imbalanced-learn

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Assuming your DataFrame is named 'df' and has been preprocessed
# Split data into features and labels
X = Original_df[feature_columns]
y = Original_df[LABEL_COLUMNS].values.ravel()  # Flatten to 1D array for consistency

# Apply preprocessing to the data
X_preprocessed = preprocessor.fit_transform(X)

# Initialize SMOTE for oversampling
smote = SMOTE(random_state=42)

# Apply SMOTE to the data
X_resampled, y_resampled = smote.fit_resample(X_preprocessed, y)

In [ ]:
print("Class distribution after oversampling:", Counter(y_resampled))

# Create a count plot for the 'FRAUD_INDICATOR' column after oversampling
plt.figure(figsize=(8, 6))
sns.countplot(data=pd.DataFrame({'FRAUD_INDICATOR': y_resampled}), x='FRAUD_INDICATOR', palette='viridis')
plt.title('Count Plot of Fraud Indicator (After Oversampling)')
plt.xlabel('Fraud Indicator')
plt.ylabel('Count')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Retraining Logistic regression using SAMPLED Data

model = LogisticRegression()

# Train the model on the training data
model.fit(X_resampled, y_resampled)

# Apply preprocessing to the testing data
X_test_preprocessed = preprocessor.transform(X_test)

# Make predictions on the testing data
y_pred = model.predict(X_test_preprocessed)

# Calculate and print various metrics to evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print("Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
model = LogisticRegression()

# Define a range of hyperparameters to search
param_grid = {
    'penalty': ['l1', 'l2'],  # Regularization type
    'C': np.logspace(-3, 3, 7),  # Inverse of regularization strength (smaller values for stronger regularization)
    'solver': ['liblinear'],  # Solver for l1 regularization
}

# Create a grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_resampled, y_resampled)

# Get the best hyperparameters and corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Evaluate the best model on the resampled data
y_pred = best_model.predict(X_resampled)

# Calculate and print various metrics to evaluate the model's performance on the resampled data
accuracy = accuracy_score(y_resampled, y_pred)
precision = precision_score(y_resampled, y_pred)
recall = recall_score(y_resampled, y_pred)
f1 = f1_score(y_resampled, y_pred)
confusion = confusion_matrix(y_resampled, y_pred)

print("Model Evaluation Metrics on Resampled Data:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Define the Random Forest Classifier model
rf_model = RandomForestClassifier(random_state=42)

# Define a range of hyperparameters to search
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}

# Create a grid search with cross-validation
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Fit the grid search to the resampled data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and corresponding model
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Train the best model on the training data
best_rf_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = best_rf_model.predict(X_test)

# Calculate and print various metrics to evaluate the best model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print("Best Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)